In [1]:
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

import fastf1 as ff1
import numpy as np
import matplotlib as mpl

from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection

ff1.Cache.enable_cache('./cache') 

In [129]:
# aggregate weather data per race
def agg_weather(weather_data):
    col_agg = {
    'AirTemp':'average',
    'Humidity':'average',
    'Pressure':'average',
    'Rainfall':'max',
    'TrackTemp':'average',
    'WindSpeed':'average'}
    weather_agg = weather_data[['AirTemp','Humidity','Pressure','Rainfall','TrackTemp','WindSpeed']].agg(col_agg,axis=0)
    return weather_agg 

# aggregate lap data according to stints (start/pit stop to pit stop/end)
def agg_tyre(lap_data):
    col_agg = {'TyreLife':'max'}
    lap_agg = lap_data[['Compound','TyreLife','Team','Driver','Stint']].groupby(['Team','Driver','Stint','Compound']).agg(col_agg,axis=0).reset_index()
    return lap_agg 

In [60]:
%%time
# to scrape
# get number of rounds from event schedule
# for each round filter conventional races (exclude training and sprint), get race session (not practice or qualifying)
# for each session, load session data, process and save aggregated value eg rainfall. no header so need to add yourself
weather_df = pd.DataFrame()

for year in range(2018,2021):
    event_schedule = ff1.get_event_schedule(year)
    event_schedule = event_schedule[event_schedule['EventFormat']=='conventional']
    rounds = event_schedule['RoundNumber']

    for roundnum in rounds:
        print("extracting " + str(year) + " " + str(roundnum))
        session = ff1.get_session(year,roundnum,'Race')
        session.load()

        # # weather
        # weather = session.weather_data
        # weather_row = pd.DataFrame(agg_weather(weather)).T
        # # add identifying columns
        # weather_row['round'] = roundnum
        # weather_row['year'] = year
        # weather_row.to_csv('./data/weather.csv',index=False,header=False,mode='a')

        # # tyres
        # laps = session.laps
        # tyre_row = agg_tyre(laps)
        # tyre_row['round'] = roundnum
        # tyre_row['year'] = year
        # tyre_row.to_csv('./data/tyrelife.csv',index=False,header=False,mode='a')

        # telemetry
        unq_drivers = list(set(session.laps.Driver))
        for driver in unq_drivers:
            try:
                tele = session.laps.pick_driver(driver).pick_fastest().telemetry
                tele['round'] = roundnum
                tele['year'] = year
                tele['driver'] = driver
                tele.to_csv('./data/telemetry.csv',index=False,header=False,mode='a')
            except:
                # some drivers may not have lap data
                pass


core           INFO 	Loading data for Australian Grand Prix - Race [v2.2.9]
api            INFO 	Using cached data for driver_info


extracting 2018 1


api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	No cached data found for position_data. Loading data...
api            INFO 	Fetching position data...
core        WARNING 	Failed to load telemetry data!
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['5', '44', '7', '3', '14', '33', '27', '77', '2', '55', '11', '31', '16', '18', '28', '8', '20', '10', '9', '35']
core           INFO 	Loading data for Bahrain Grand Prix - Race [v2.2.9]
api            INFO 	Using cached data for driver_info


extracting 2018 2


api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	No cached data found for car_data. Loading data...
api            INFO 	Fetching car data...
core        WARNING 	Failed to load telemetry data!
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['5', '77', '44', '10', '20', '27', '14', '2', '9', '31', '55', '16', '8', '18', '35', '11', '28', '7', '33', '3']
/Users/blossomtang/opt/anaconda3/envs/py39/lib/python3.9/site-packages/fastf1/core.py:2165: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
 

...

extracting 2020 16


api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['11', '31', '18', '55', '3', '23', '26', '77', '63', '4', '10', '5', '99', '7', '20', '89', '51', '6', '33', '16']
/Users/blossomtang/opt/anaconda3/envs/py39/lib/python3.9/site-packages/fastf1/core.py:2165: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  super().__init__(*args, **kwargs)
/Users/blossomtang/op

extracting 2020 17


api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['33', '77', '44', '23', '4', '55', '3', '10', '31', '18', '26', '7', '16', '5', '63', '99', '6', '20', '51', '11']


CPU times: user 16min 53s, sys: 16.5 s, total: 17min 9s
Wall time: 18min 9s


In [142]:
event_schedule = ff1.get_event_schedule(2021)
event_schedule.head(3)

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session2,Session2Date,Session3,Session3Date,Session4,Session4Date,Session5,Session5Date,F1ApiSupport
0,0,Bahrain,Sakhir,FORMULA 1 ARAMCO PRE-SEASON TESTING 2021,2021-03-14,Pre-Season Test,testing,Practice 1,2021-03-12,Practice 2,2021-03-13,Practice 3,2021-03-14,None,NaT,None,NaT,True
1,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2021,2021-03-28,Bahrain Grand Prix,conventional,Practice 1,2021-03-26,Practice 2,2021-03-26,Practice 3,2021-03-27,Qualifying,2021-03-27,Race,2021-03-28,True
2,2,Italy,Imola,FORMULA 1 PIRELLI GRAN PREMIO DEL MADE IN ITAL...,2021-04-18,Emilia Romagna Grand Prix,conventional,Practice 1,2021-04-16,Practice 2,2021-04-16,Practice 3,2021-04-17,Qualifying,2021-04-17,Race,2021-04-18,True


In [3]:
# read ergast data
circuits = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/circuits.csv')
constructor_results = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/constructor_results.csv')
constructor_standings = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/constructor_standings.csv')
constructors = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/constructors.csv')
driver_standings = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/driver_standings.csv')
drivers = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/drivers.csv')
lap_times = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/lap_times.csv')
pit_stops = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/pit_stops.csv')
qualifying = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/qualifying.csv')
races = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/races.csv')
results = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/results.csv')
seasons = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/seasons.csv')
status = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-09-07/status.csv')